# Lab 1.5: Professional Command-Line Interfaces
**Module 1 | Duration: 2-3 hours | Bridge: Basic Scripts → Professional CLI Tools**

## Learning Objectives
- Build professional command-line interfaces using argparse
- Implement CLI patterns that follow Unix conventions
- Create user-friendly help systems and error messages
- Add interactive features and configuration management
- Integrate CLI tools into automation workflows

## Scenario: Sarah's Power User Transformation

Sarah has been using your refactored AI assistant and loves the improvements! Now she has a new request:

*"This is fantastic, but I'm spending a lot of time in the terminal for my work. Could you make the assistant work like the professional CLI tools I use? I want to pipe outputs, use it in scripts, and have quick one-liner commands. Think of how `git`, `pip`, or `docker` work – that's what I need for my AI assistant."*

Your mission: Transform the assistant into a professional CLI tool that integrates seamlessly with Sarah's development workflow.

---

## Part 1: Understanding Professional CLI Patterns

Before we build, let's analyze what makes CLI tools professional and user-friendly.

In [ ]:
# Let's examine professional CLI patterns from popular tools
import subprocess
import os

print("🔍 Analyzing Professional CLI Patterns")
print("="*50)

# Examples of professional CLI help output
examples = {
    "git": "git --help | head -15",
    "pip": "pip --help | head -10", 
    "python": "python --help | head -10"
}

print("Professional CLI tools share common patterns:")
print("\n1. Clear, hierarchical help systems")
print("2. Consistent argument patterns")
print("3. Multiple interface modes (interactive vs batch)")
print("4. Proper exit codes for automation")
print("5. Configuration file support")
print("6. Stdin/stdout/stderr handling")

print("\n📋 CLI Design Principles:")
print("• Be explicit: Clear commands and options")
print("• Be helpful: Good help text and error messages")
print("• Be consistent: Follow established conventions")
print("• Be efficient: Support power user workflows")

### 🧪 Task 1.1: CLI Pattern Analysis

Think about CLI tools you use regularly. Complete the analysis below:

**Your Analysis** (Replace this with your findings):

| Tool | Commands/Features You Use | What Makes It User-Friendly |
|------|---------------------------|-----------------------------|
| git | `git status`, `git commit -m "..."` | _[What makes git easy to use?]_ |
| pip | `pip install package` | _[What makes pip intuitive?]_ |
| _[Your tool]_ | _[Commands you use]_ | _[What you like about it]_ |

**Key Patterns for Sarah's AI Assistant**:
1. _[Pattern #1 to implement]_
2. _[Pattern #2 to implement]_
3. _[Pattern #3 to implement]_

---

## Part 2: Building the Argument Parser Foundation

Let's start by creating a robust argument parsing system using Python's `argparse` module.

In [ ]:
# cli_parser.py - Professional Argument Parsing System

import argparse
import sys
from typing import List, Optional

class AIAssistantCLI:
    """
    Professional CLI interface for AI Assistant.
    Follows Unix conventions and provides comprehensive help.
    """
    
    def __init__(self):
        self.parser = self._create_parser()
    
    def _create_parser(self) -> argparse.ArgumentParser:
        """
        Create the main argument parser with subcommands.
        """
        parser = argparse.ArgumentParser(
            prog='ai-assistant',
            description='Professional AI Assistant - Your intelligent command-line companion',
            epilog="""Examples:
  %(prog)s chat "What is machine learning?"
  %(prog)s chat --file input.txt --output response.txt
  %(prog)s interactive --model gpt-4
  %(prog)s config --set model gpt-4
  echo "Explain this code" | %(prog)s chat --stdin
  
For more help on a specific command, use:
  %(prog)s <command> --help""",
            formatter_class=argparse.RawDescriptionHelpFormatter
        )
        
        # Global options
        parser.add_argument(
            '--version',
            action='version',
            version='AI Assistant 2.0.0'
        )
        
        parser.add_argument(
            '--config',
            help='Configuration file path (default: ~/.ai-assistant.conf)',
            default='~/.ai-assistant.conf'
        )
        
        parser.add_argument(
            '--verbose', '-v',
            action='store_true',
            help='Enable verbose output'
        )
        
        parser.add_argument(
            '--quiet', '-q',
            action='store_true',
            help='Suppress non-essential output'
        )
        
        # Create subcommands
        subparsers = parser.add_subparsers(
            dest='command',
            help='Available commands',
            metavar='COMMAND'
        )
        
        # Chat command (main functionality)
        self._add_chat_parser(subparsers)
        
        # Interactive command
        self._add_interactive_parser(subparsers)
        
        # Configuration command
        self._add_config_parser(subparsers)
        
        # History command
        self._add_history_parser(subparsers)
        
        return parser
    
    def _add_chat_parser(self, subparsers):
        """
        Add chat command parser.
        """
        chat_parser = subparsers.add_parser(
            'chat',
            help='Send a message to the AI assistant',
            description='Send a message to the AI and get a response',
            epilog="""Examples:
  ai-assistant chat "What is Python?"
  ai-assistant chat --file question.txt --output answer.txt
  echo "Explain this" | ai-assistant chat --stdin""",
            formatter_class=argparse.RawDescriptionHelpFormatter
        )
        
        # Message input options (mutually exclusive)
        input_group = chat_parser.add_mutually_exclusive_group(required=True)
        
        input_group.add_argument(
            'message',
            nargs='?',
            help='Message to send to the AI'
        )
        
        input_group.add_argument(
            '--file', '-f',
            help='Read message from file'
        )
        
        input_group.add_argument(
            '--stdin',
            action='store_true',
            help='Read message from standard input'
        )
        
        # Output options
        chat_parser.add_argument(
            '--output', '-o',
            help='Write response to file (default: stdout)'
        )
        
        # AI model options
        chat_parser.add_argument(
            '--model', '-m',
            choices=['gpt-3.5-turbo', 'gpt-4', 'claude-3-sonnet', 'claude-3-opus'],
            help='AI model to use (overrides config)'
        )
        
        chat_parser.add_argument(
            '--temperature', '-t',
            type=float,
            metavar='TEMP',
            help='Response temperature (0.0-2.0, default from config)'
        )
        
        chat_parser.add_argument(
            '--max-tokens',
            type=int,
            metavar='N',
            help='Maximum tokens in response (default from config)'
        )
        
        # Context options
        chat_parser.add_argument(
            '--no-context',
            action='store_true',
            help='Ignore conversation history'
        )
        
        chat_parser.add_argument(
            '--system-prompt',
            help='Custom system prompt for this conversation'
        )
    
    def _add_interactive_parser(self, subparsers):
        """
        Add interactive command parser.
        """
        interactive_parser = subparsers.add_parser(
            'interactive',
            help='Start interactive chat session',
            description='Start an interactive chat session with the AI'
        )
        
        interactive_parser.add_argument(
            '--model', '-m',
            choices=['gpt-3.5-turbo', 'gpt-4', 'claude-3-sonnet', 'claude-3-opus'],
            help='AI model to use'
        )
        
        interactive_parser.add_argument(
            '--save-session',
            help='Save conversation to specified file'
        )
    
    def _add_config_parser(self, subparsers):
        """
        Add configuration command parser.
        """
        config_parser = subparsers.add_parser(
            'config',
            help='Manage configuration settings',
            description='View and modify configuration settings'
        )
        
        config_actions = config_parser.add_mutually_exclusive_group(required=True)
        
        config_actions.add_argument(
            '--list',
            action='store_true',
            help='List all configuration settings'
        )
        
        config_actions.add_argument(
            '--get',
            metavar='KEY',
            help='Get value of specific setting'
        )
        
        config_actions.add_argument(
            '--set',
            nargs=2,
            metavar=('KEY', 'VALUE'),
            help='Set configuration value'
        )
        
        config_actions.add_argument(
            '--reset',
            action='store_true',
            help='Reset to default configuration'
        )
    
    def _add_history_parser(self, subparsers):
        """
        Add history command parser.
        """
        history_parser = subparsers.add_parser(
            'history',
            help='Manage conversation history',
            description='View and manage conversation history'
        )
        
        history_actions = history_parser.add_mutually_exclusive_group(required=True)
        
        history_actions.add_argument(
            '--list',
            action='store_true',
            help='List recent conversations'
        )
        
        history_actions.add_argument(
            '--search',
            metavar='QUERY',
            help='Search conversation history'
        )
        
        history_actions.add_argument(
            '--export',
            metavar='FILE',
            help='Export history to file'
        )
        
        history_actions.add_argument(
            '--clear',
            action='store_true',
            help='Clear conversation history'
        )
        
        history_parser.add_argument(
            '--limit', '-n',
            type=int,
            default=10,
            help='Number of conversations to show (default: 10)'
        )
    
    def parse_args(self, args: Optional[List[str]] = None):
        """
        Parse command line arguments.
        """
        return self.parser.parse_args(args)
    
    def print_help(self):
        """
        Print help message.
        """
        self.parser.print_help()

# Test the parser
print("✓ Professional CLI parser created!")
print("This provides a comprehensive command-line interface.")

# Demonstrate the help system
cli = AIAssistantCLI()
print("\n📋 Available commands:")
print("• chat - Send messages to AI")
print("• interactive - Start interactive session")
print("• config - Manage settings")
print("• history - View/manage conversation history")

### 🧪 Task 1.2: Testing the CLI Parser

Let's test our argument parser with various command combinations:

In [ ]:
# Test the CLI parser with different argument combinations

print("🧪 Testing CLI Argument Parsing")
print("="*40)

# Test cases that should work
test_cases = [
    ['chat', 'Hello, AI!'],
    ['chat', '--file', 'input.txt'],
    ['chat', '--stdin', '--output', 'response.txt'],
    ['interactive', '--model', 'gpt-4'],
    ['config', '--list'],
    ['config', '--set', 'model', 'gpt-4'],
    ['history', '--list', '--limit', '5'],
    ['--verbose', 'chat', 'Test message']
]

cli = AIAssistantCLI()

for i, test_args in enumerate(test_cases, 1):
    try:
        args = cli.parse_args(test_args)
        print(f"✓ Test {i}: {' '.join(test_args)} - PASSED")
        print(f"  Command: {args.command}")
        if hasattr(args, 'message') and args.message:
            print(f"  Message: {args.message}")
        if hasattr(args, 'verbose') and args.verbose:
            print(f"  Verbose mode enabled")
        print()
    except SystemExit:
        print(f"❌ Test {i}: {' '.join(test_args)} - FAILED")
    except Exception as e:
        print(f"❌ Test {i}: {' '.join(test_args)} - ERROR: {e}")

print("\n📋 Command Structure Validation Complete")

---

## Part 3: Configuration Management System

Professional CLI tools need robust configuration management. Let's build a system that handles config files and environment variables.

In [ ]:
# cli_config.py - Professional Configuration Management

import os
import json
import configparser
from pathlib import Path
from typing import Dict, Any, Optional
from dataclasses import dataclass, asdict

@dataclass
class CLIConfig:
    """
    Configuration settings for the CLI application.
    """
    # AI settings
    api_key: str = ""
    model: str = "gpt-3.5-turbo"
    temperature: float = 0.7
    max_tokens: int = 1000
    system_prompt: str = "You are a helpful AI assistant."
    
    # Application settings
    conversation_file: str = "~/.ai-assistant-history.json"
    auto_save: bool = True
    max_history: int = 20
    
    # CLI settings
    default_output_format: str = "text"
    color_output: bool = True
    verbose: bool = False
    
class ConfigManager:
    """
    Manages configuration loading, saving, and validation.
    Handles multiple configuration sources with proper precedence.
    """
    
    def __init__(self, config_path: Optional[str] = None):
        self.config_path = Path(config_path or "~/.ai-assistant.conf").expanduser()
        self.config = CLIConfig()
        
    def load_config(self) -> CLIConfig:
        """
        Load configuration from multiple sources with proper precedence:
        1. Default values
        2. Configuration file
        3. Environment variables
        4. Command line arguments (handled separately)
        """
        # Start with defaults
        config = CLIConfig()
        
        # Load from config file if it exists
        if self.config_path.exists():
            file_config = self._load_from_file()
            if file_config:
                config = self._merge_configs(config, file_config)
        
        # Override with environment variables
        env_config = self._load_from_environment()
        config = self._merge_configs(config, env_config)
        
        self.config = config
        return config
    
    def _load_from_file(self) -> Optional[Dict[str, Any]]:
        """
        Load configuration from INI file.
        """
        try:
            parser = configparser.ConfigParser()
            parser.read(self.config_path)
            
            config = {}
            
            # Parse each section
            for section_name in parser.sections():
                section = parser[section_name]
                for key, value in section.items():
                    # Convert string values to appropriate types
                    config[key] = self._convert_value(value)
            
            return config
            
        except Exception as e:
            print(f"Warning: Could not load config file {self.config_path}: {e}")
            return None
    
    def _load_from_environment(self) -> Dict[str, Any]:
        """
        Load configuration from environment variables.
        """
        env_mapping = {
            'AI_ASSISTANT_API_KEY': 'api_key',
            'AI_ASSISTANT_MODEL': 'model',
            'AI_ASSISTANT_TEMPERATURE': 'temperature',
            'AI_ASSISTANT_MAX_TOKENS': 'max_tokens',
            'AI_ASSISTANT_SYSTEM_PROMPT': 'system_prompt',
            'AI_ASSISTANT_CONVERSATION_FILE': 'conversation_file',
            'AI_ASSISTANT_AUTO_SAVE': 'auto_save',
            'AI_ASSISTANT_MAX_HISTORY': 'max_history',
            'AI_ASSISTANT_VERBOSE': 'verbose',
            'AI_ASSISTANT_COLOR': 'color_output'
        }
        
        config = {}
        for env_var, config_key in env_mapping.items():
            value = os.getenv(env_var)
            if value is not None:
                config[config_key] = self._convert_value(value)
        
        return config
    
    def _convert_value(self, value: str) -> Any:
        """
        Convert string value to appropriate Python type.
        """
        # Boolean values
        if value.lower() in ('true', 'yes', '1', 'on'):
            return True
        elif value.lower() in ('false', 'no', '0', 'off'):
            return False
        
        # Numeric values
        try:
            if '.' in value:
                return float(value)
            else:
                return int(value)
        except ValueError:
            pass
        
        # String value
        return value
    
    def _merge_configs(self, base: CLIConfig, override: Dict[str, Any]) -> CLIConfig:
        """
        Merge configuration dictionaries, with override taking precedence.
        """
        config_dict = asdict(base)
        
        for key, value in override.items():
            if key in config_dict:
                config_dict[key] = value
        
        return CLIConfig(**config_dict)
    
    def save_config(self, config: Optional[CLIConfig] = None) -> bool:
        """
        Save configuration to file.
        """
        config = config or self.config
        
        try:
            # Ensure directory exists
            self.config_path.parent.mkdir(parents=True, exist_ok=True)
            
            # Create configparser object
            parser = configparser.ConfigParser()
            
            # Add sections
            parser.add_section('ai')
            parser.set('ai', 'model', config.model)
            parser.set('ai', 'temperature', str(config.temperature))
            parser.set('ai', 'max_tokens', str(config.max_tokens))
            parser.set('ai', 'system_prompt', config.system_prompt)
            
            parser.add_section('application')
            parser.set('application', 'conversation_file', config.conversation_file)
            parser.set('application', 'auto_save', str(config.auto_save))
            parser.set('application', 'max_history', str(config.max_history))
            
            parser.add_section('cli')
            parser.set('cli', 'default_output_format', config.default_output_format)
            parser.set('cli', 'color_output', str(config.color_output))
            parser.set('cli', 'verbose', str(config.verbose))
            
            # Write to file
            with open(self.config_path, 'w') as f:
                parser.write(f)
            
            return True
            
        except Exception as e:
            print(f"Error saving config to {self.config_path}: {e}")
            return False
    
    def get_setting(self, key: str) -> Any:
        """
        Get a specific configuration setting.
        """
        return getattr(self.config, key, None)
    
    def set_setting(self, key: str, value: Any) -> bool:
        """
        Set a specific configuration setting.
        """
        if hasattr(self.config, key):
            setattr(self.config, key, value)
            return True
        return False
    
    def list_settings(self) -> Dict[str, Any]:
        """
        List all configuration settings.
        """
        return asdict(self.config)
    
    def reset_to_defaults(self) -> bool:
        """
        Reset configuration to default values.
        """
        self.config = CLIConfig()
        return self.save_config()
    
    def validate_config(self) -> List[str]:
        """
        Validate configuration and return list of issues.
        """
        issues = []
        
        # Check required settings
        if not self.config.api_key:
            issues.append("API key is required. Set AI_ASSISTANT_API_KEY environment variable or use 'ai-assistant config --set api_key YOUR_KEY'")
        
        # Validate numeric ranges
        if not 0.0 <= self.config.temperature <= 2.0:
            issues.append("Temperature must be between 0.0 and 2.0")
        
        if self.config.max_tokens <= 0:
            issues.append("Max tokens must be positive")
        
        if self.config.max_history <= 0:
            issues.append("Max history must be positive")
        
        return issues

print("✓ Configuration management system created!")
print("This handles config files, environment variables, and validation.")

### 🧪 Task 1.3: Testing Configuration Management

Let's test the configuration system:

In [ ]:
# Test configuration management

print("🧪 Testing Configuration Management")
print("="*40)

# Test 1: Default configuration
config_manager = ConfigManager("/tmp/test-ai-assistant.conf")
config = config_manager.load_config()

print("✓ Test 1: Default configuration loaded")
print(f"  Model: {config.model}")
print(f"  Temperature: {config.temperature}")
print(f"  Max tokens: {config.max_tokens}")

# Test 2: Environment variable override
os.environ['AI_ASSISTANT_MODEL'] = 'gpt-4'
os.environ['AI_ASSISTANT_TEMPERATURE'] = '0.9'
os.environ['AI_ASSISTANT_VERBOSE'] = 'true'

config = config_manager.load_config()
print("\n✓ Test 2: Environment variable override")
print(f"  Model: {config.model}")
print(f"  Temperature: {config.temperature}")
print(f"  Verbose: {config.verbose}")

# Test 3: Configuration validation
issues = config_manager.validate_config()
print("\n✓ Test 3: Configuration validation")
if issues:
    print("  Issues found:")
    for issue in issues:
        print(f"    - {issue}")
else:
    print("  No issues found")

# Test 4: Settings management
print("\n✓ Test 4: Settings management")
success = config_manager.set_setting('model', 'claude-3-sonnet')
print(f"  Set model: {success}")
print(f"  Current model: {config_manager.get_setting('model')}")

# Clean up test environment
for key in ['AI_ASSISTANT_MODEL', 'AI_ASSISTANT_TEMPERATURE', 'AI_ASSISTANT_VERBOSE']:
    if key in os.environ:
        del os.environ[key]

print("\n📋 Configuration system validation complete")

---

## Part 4: Command Implementation

Now let's implement the actual command handlers that tie everything together.

In [ ]:
# cli_commands.py - Command Implementation Layer

import sys
import os
from pathlib import Path
from typing import Optional, List
import json

class CLICommands:
    """
    Implementation of CLI commands.
    Each command follows Unix conventions for input/output and exit codes.
    """
    
    def __init__(self, config_manager: ConfigManager):
        self.config_manager = config_manager
        self.config = config_manager.config
        
        # Initialize AI assistant components (would normally import from previous modules)
        self.ai_assistant = None  # Would be AIAssistant(config)
    
    def handle_chat(self, args) -> int:
        """
        Handle the chat command.
        Returns exit code (0 = success, non-zero = error).
        """
        try:
            # Get input message
            message = self._get_input_message(args)
            if not message:
                self._error("No input message provided")
                return 1
            
            # Apply command-line overrides to config
            runtime_config = self._apply_runtime_overrides(args)
            
            # Get AI response (simulated for this lab)
            if args.verbose:
                self._info(f"Sending message to {runtime_config.model}...")
            
            # Simulate AI response
            response = f"[AI Response using {runtime_config.model}] I received your message: '{message[:50]}...' and would respond appropriately."
            
            # Output response
            self._output_response(response, args.output)
            
            if args.verbose:
                self._info("Response generated successfully")
            
            return 0
            
        except KeyboardInterrupt:
            self._error("\nOperation cancelled by user")
            return 130
        except Exception as e:
            self._error(f"Chat command failed: {e}")
            return 1
    
    def handle_interactive(self, args) -> int:
        """
        Handle the interactive command.
        """
        try:
            self._info("Starting interactive mode...")
            self._info("Type 'quit' or 'exit' to end the session.")
            self._info("Type '/help' for available commands.")
            print()
            
            session_history = []
            
            while True:
                try:
                    user_input = input("You: ").strip()
                    
                    if not user_input:
                        continue
                    
                    if user_input.lower() in ['quit', 'exit']:
                        break
                    
                    if user_input == '/help':
                        self._show_interactive_help()
                        continue
                    
                    # Simulate AI response
                    response = f"[Interactive AI] Response to: {user_input}"
                    print(f"AI: {response}")
                    print()
                    
                    session_history.append({"user": user_input, "ai": response})
                    
                except KeyboardInterrupt:
                    print("\n\nSession ended by user.")
                    break
            
            # Save session if requested
            if args.save_session and session_history:
                self._save_session(session_history, args.save_session)
            
            self._info(f"Interactive session ended. {len(session_history)} exchanges.")
            return 0
            
        except Exception as e:
            self._error(f"Interactive mode failed: {e}")
            return 1
    
    def handle_config(self, args) -> int:
        """
        Handle the config command.
        """
        try:
            if args.list:
                settings = self.config_manager.list_settings()
                print("Current configuration:")
                for key, value in settings.items():
                    if key == 'api_key' and value:
                        # Mask API key for security
                        masked_value = value[:8] + "*" * (len(value) - 8)
                        print(f"  {key}: {masked_value}")
                    else:
                        print(f"  {key}: {value}")
                return 0
            
            elif args.get:
                value = self.config_manager.get_setting(args.get)
                if value is not None:
                    if args.get == 'api_key' and value:
                        print(f"{args.get}: [MASKED]")
                    else:
                        print(f"{args.get}: {value}")
                    return 0
                else:
                    self._error(f"Unknown setting: {args.get}")
                    return 1
            
            elif args.set:
                key, value = args.set
                success = self.config_manager.set_setting(key, value)
                if success:
                    # Save to file
                    if self.config_manager.save_config():
                        self._info(f"Setting {key} updated and saved")
                        return 0
                    else:
                        self._error("Failed to save configuration")
                        return 1
                else:
                    self._error(f"Unknown setting: {key}")
                    return 1
            
            elif args.reset:
                if self.config_manager.reset_to_defaults():
                    self._info("Configuration reset to defaults")
                    return 0
                else:
                    self._error("Failed to reset configuration")
                    return 1
            
        except Exception as e:
            self._error(f"Config command failed: {e}")
            return 1
    
    def handle_history(self, args) -> int:
        """
        Handle the history command.
        """
        try:
            if args.list:
                print(f"Recent conversations (last {args.limit}):")
                # Simulate conversation history
                for i in range(min(args.limit, 3)):
                    print(f"  {i+1}. [2024-01-{20+i}] User: Sample question {i+1}")
                    print(f"      AI: Sample response {i+1}")
                return 0
            
            elif args.search:
                print(f"Searching for: '{args.search}'")
                print("No matching conversations found (simulated)")
                return 0
            
            elif args.export:
                print(f"Exporting history to {args.export}...")
                # Simulate export
                with open(args.export, 'w') as f:
                    json.dump([{"sample": "conversation"}], f, indent=2)
                self._info(f"History exported to {args.export}")
                return 0
            
            elif args.clear:
                confirm = input("Are you sure you want to clear all history? (y/N): ")
                if confirm.lower() == 'y':
                    self._info("Conversation history cleared")
                else:
                    self._info("Operation cancelled")
                return 0
            
        except Exception as e:
            self._error(f"History command failed: {e}")
            return 1
    
    def _get_input_message(self, args) -> Optional[str]:
        """
        Get input message from various sources.
        """
        if args.message:
            return args.message
        elif args.file:
            try:
                with open(args.file, 'r') as f:
                    return f.read().strip()
            except Exception as e:
                self._error(f"Could not read file {args.file}: {e}")
                return None
        elif args.stdin:
            try:
                return sys.stdin.read().strip()
            except Exception as e:
                self._error(f"Could not read from stdin: {e}")
                return None
        return None
    
    def _apply_runtime_overrides(self, args):
        """
        Apply command-line argument overrides to configuration.
        """
        runtime_config = self.config
        
        if hasattr(args, 'model') and args.model:
            runtime_config.model = args.model
        if hasattr(args, 'temperature') and args.temperature is not None:
            runtime_config.temperature = args.temperature
        if hasattr(args, 'max_tokens') and args.max_tokens:
            runtime_config.max_tokens = args.max_tokens
        if hasattr(args, 'system_prompt') and args.system_prompt:
            runtime_config.system_prompt = args.system_prompt
        
        return runtime_config
    
    def _output_response(self, response: str, output_file: Optional[str]):
        """
        Output response to stdout or file.
        """
        if output_file:
            with open(output_file, 'w') as f:
                f.write(response)
        else:
            print(response)
    
    def _save_session(self, history: List, filename: str):
        """
        Save interactive session to file.
        """
        try:
            with open(filename, 'w') as f:
                json.dump(history, f, indent=2)
            self._info(f"Session saved to {filename}")
        except Exception as e:
            self._error(f"Could not save session: {e}")
    
    def _show_interactive_help(self):
        """
        Show help for interactive mode.
        """
        print("Interactive mode commands:")
        print("  /help - Show this help")
        print("  quit, exit - End session")
        print("  Just type your message to chat!")
        print()
    
    def _info(self, message: str):
        """
        Print info message to stderr.
        """
        if not self.config.verbose:
            return
        print(f"ℹ️  {message}", file=sys.stderr)
    
    def _error(self, message: str):
        """
        Print error message to stderr.
        """
        print(f"❌ Error: {message}", file=sys.stderr)

print("✓ CLI command handlers created!")
print("These implement professional CLI patterns with proper error handling.")

---

## Part 5: Complete CLI Application

Now let's put it all together into a complete CLI application.

In [ ]:
# ai_assistant_cli.py - Complete CLI Application

#!/usr/bin/env python3
"""
AI Assistant CLI - Professional command-line interface for AI interactions.

This tool provides a professional CLI for interacting with AI models,
following Unix conventions and supporting automation workflows.

Usage:
    ai-assistant chat "Your message here"
    ai-assistant interactive
    ai-assistant config --list
    ai-assistant history --list
"""

import sys
import os
from pathlib import Path

class AIAssistantCLIApp:
    """
    Main CLI application class.
    Orchestrates argument parsing, configuration, and command execution.
    """
    
    def __init__(self):
        self.cli_parser = AIAssistantCLI()
        self.config_manager = None
        self.commands = None
    
    def run(self, argv: list = None) -> int:
        """
        Main entry point for the CLI application.
        
        Args:
            argv: Command line arguments (defaults to sys.argv[1:])
            
        Returns:
            Exit code (0 = success, non-zero = error)
        """
        try:
            # Parse arguments
            args = self.cli_parser.parse_args(argv)
            
            # Initialize configuration
            self.config_manager = ConfigManager(args.config)
            config = self.config_manager.load_config()
            
            # Apply global argument overrides
            if args.verbose:
                config.verbose = True
            if args.quiet:
                config.verbose = False
            
            # Validate configuration
            issues = self.config_manager.validate_config()
            if issues and args.command == 'chat':
                # Only require valid config for commands that need AI
                for issue in issues:
                    print(f"❌ Configuration issue: {issue}", file=sys.stderr)
                return 1
            
            # Initialize command handlers
            self.commands = CLICommands(self.config_manager)
            
            # Route to appropriate command handler
            if args.command == 'chat':
                return self.commands.handle_chat(args)
            elif args.command == 'interactive':
                return self.commands.handle_interactive(args)
            elif args.command == 'config':
                return self.commands.handle_config(args)
            elif args.command == 'history':
                return self.commands.handle_history(args)
            else:
                # No command specified, show help
                self.cli_parser.print_help()
                return 1
        
        except KeyboardInterrupt:
            print("\n\nOperation cancelled by user.", file=sys.stderr)
            return 130
        except SystemExit as e:
            # argparse calls sys.exit() for help/version
            return e.code
        except Exception as e:
            print(f"❌ Unexpected error: {e}", file=sys.stderr)
            if config.verbose if 'config' in locals() else False:
                import traceback
                traceback.print_exc()
            return 1

def main():
    """
    Entry point for the command-line script.
    """
    app = AIAssistantCLIApp()
    exit_code = app.run()
    sys.exit(exit_code)

if __name__ == '__main__':
    main()

print("✓ Complete CLI application created!")
print("This is a production-ready command-line tool.")

### 🧪 Task 1.4: Testing the Complete CLI Application

Let's test our complete CLI application with various scenarios:

In [ ]:
# Test the complete CLI application

print("🧪 Testing Complete CLI Application")
print("="*50)

# Test scenarios
test_scenarios = [
    {
        'name': 'Help Command',
        'args': ['--help'],
        'expected_exit': 0
    },
    {
        'name': 'Version Command',
        'args': ['--version'],
        'expected_exit': 0
    },
    {
        'name': 'Chat with Message',
        'args': ['chat', 'Hello, AI!'],
        'expected_exit': 1  # Will fail due to missing API key
    },
    {
        'name': 'Config List',
        'args': ['config', '--list'],
        'expected_exit': 0
    },
    {
        'name': 'History List',
        'args': ['history', '--list'],
        'expected_exit': 0
    }
]

app = AIAssistantCLIApp()
passed_tests = 0
total_tests = len(test_scenarios)

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n📋 Test {i}: {scenario['name']}")
    print(f"   Command: ai-assistant {' '.join(scenario['args'])}")
    
    try:
        # Capture output for testing
        import io
        from contextlib import redirect_stdout, redirect_stderr
        
        stdout_capture = io.StringIO()
        stderr_capture = io.StringIO()
        
        try:
            with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture):
                exit_code = app.run(scenario['args'])
        except SystemExit as e:
            exit_code = e.code
        
        stdout_output = stdout_capture.getvalue()
        stderr_output = stderr_capture.getvalue()
        
        # Check results
        if scenario['name'] in ['Help Command', 'Version Command']:
            # These should produce output and exit cleanly
            if exit_code in [0, None] and (stdout_output or stderr_output):
                print(f"   ✓ PASSED - Command executed successfully")
                passed_tests += 1
            else:
                print(f"   ❌ FAILED - Expected help/version output")
        else:
            # Regular commands
            if exit_code == scenario['expected_exit']:
                print(f"   ✓ PASSED - Exit code {exit_code} as expected")
                passed_tests += 1
            else:
                print(f"   ❌ FAILED - Exit code {exit_code}, expected {scenario['expected_exit']}")
        
        # Show sample output (first few lines)
        if stdout_output:
            lines = stdout_output.split('\n')[:3]
            print(f"   Output: {lines[0][:60]}...")
        if stderr_output:
            lines = stderr_output.split('\n')[:2]
            print(f"   Error: {lines[0][:60]}...")
    
    except Exception as e:
        print(f"   ❌ FAILED - Exception: {e}")

print(f"\n🎯 Test Summary: {passed_tests}/{total_tests} tests passed")
if passed_tests == total_tests:
    print("🎉 All tests passed! CLI application is working correctly.")
else:
    print("⚠️  Some tests failed. This is expected without proper API configuration.")

---

## Part 6: Advanced CLI Features

Let's add some advanced features that make the CLI even more professional.

### Task 1.5: Choose and Implement an Advanced Feature

Choose one of these advanced features to implement:

**Advanced Feature Options:**

1. **Shell Completion**: Auto-completion for commands and options
2. **Progress Indicators**: Show progress during long AI operations  
3. **Colored Output**: Professional colored terminal output
4. **Pipe Support**: Better stdin/stdout handling for Unix pipes
5. **Plugin System**: Simple plugin architecture for extensions

**Your Choice**: _[Select one and explain why]_

**Implementation Plan**: _[Describe how you'll implement it]_

In [ ]:
# Implement your chosen advanced feature here
# Example: Colored Output System

import os
import sys

class ColoredOutput:
    """
    Professional colored terminal output with automatic color detection.
    """
    
    # ANSI color codes
    COLORS = {
        'reset': '\033[0m',
        'bold': '\033[1m',
        'dim': '\033[2m',
        'red': '\033[31m',
        'green': '\033[32m',
        'yellow': '\033[33m',
        'blue': '\033[34m',
        'magenta': '\033[35m',
        'cyan': '\033[36m',
        'white': '\033[37m'
    }
    
    def __init__(self, enabled: bool = None):
        if enabled is None:
            # Auto-detect color support
            self.enabled = self._supports_color()
        else:
            self.enabled = enabled
    
    def _supports_color(self) -> bool:
        """
        Detect if terminal supports color output.
        """
        # Check if NO_COLOR environment variable is set
        if os.getenv('NO_COLOR'):
            return False
        
        # Check if FORCE_COLOR is set
        if os.getenv('FORCE_COLOR'):
            return True
        
        # Check if stdout is a TTY
        if not hasattr(sys.stdout, 'isatty') or not sys.stdout.isatty():
            return False
        
        # Check TERM environment variable
        term = os.getenv('TERM', '')
        if term in ['dumb', 'unknown']:
            return False
        
        return True
    
    def colorize(self, text: str, color: str, bold: bool = False) -> str:
        """
        Apply color to text if color output is enabled.
        """
        if not self.enabled:
            return text
        
        codes = []
        if bold:
            codes.append(self.COLORS['bold'])
        if color in self.COLORS:
            codes.append(self.COLORS[color])
        
        if codes:
            return ''.join(codes) + text + self.COLORS['reset']
        return text
    
    def success(self, text: str) -> str:
        """Format success message."""
        return self.colorize(f"✓ {text}", 'green')
    
    def error(self, text: str) -> str:
        """Format error message."""
        return self.colorize(f"❌ {text}", 'red', bold=True)
    
    def warning(self, text: str) -> str:
        """Format warning message."""
        return self.colorize(f"⚠️  {text}", 'yellow')
    
    def info(self, text: str) -> str:
        """Format info message."""
        return self.colorize(f"ℹ️  {text}", 'cyan')
    
    def command(self, text: str) -> str:
        """Format command text."""
        return self.colorize(text, 'blue', bold=True)
    
    def highlight(self, text: str) -> str:
        """Highlight important text."""
        return self.colorize(text, 'magenta', bold=True)

# Test the colored output system
colors = ColoredOutput()

print("🎨 Testing Colored Output System")
print("="*40)
print(colors.success("This is a success message"))
print(colors.error("This is an error message"))
print(colors.warning("This is a warning message"))
print(colors.info("This is an info message"))
print(colors.command("ai-assistant chat 'Hello!'"))
print(colors.highlight("This text is highlighted"))

print(f"\nColor support detected: {colors.enabled}")
print("\n✓ Advanced feature implemented!")

---

## Part 7: CLI Usage Examples and Documentation

Let's create comprehensive usage examples that Sarah can reference.

### 📚 Professional CLI Usage Guide

**Task 1.6**: Complete this usage guide based on your CLI implementation:

#### Basic Usage Examples

```bash
# Quick question
ai-assistant chat "What is machine learning?"

# Read question from file
ai-assistant chat --file question.txt

# Save response to file
ai-assistant chat "Explain Python" --output explanation.txt

# Use specific model
ai-assistant chat "Complex question" --model gpt-4
```

#### Pipeline Integration

```bash
# Use in pipes
echo "Explain this code" | ai-assistant chat --stdin

# Process multiple files
for file in *.py; do
    echo "Reviewing $file"
    ai-assistant chat --file "$file" --output "review-$file.txt"
done

# Chain with other tools
cat code.py | ai-assistant chat --stdin | less
```

#### Configuration Management

```bash
# Set up API key
ai-assistant config --set api_key "your-key-here"

# Set default model
ai-assistant config --set model gpt-4

# View current settings
ai-assistant config --list
```

#### History Management

```bash
# View recent conversations
ai-assistant history --list

# Search conversation history
ai-assistant history --search "machine learning"

# Export conversations
ai-assistant history --export backup.json
```

#### Environment Variables

```bash
# Set API key via environment
export AI_ASSISTANT_API_KEY="your-key"

# Set default model
export AI_ASSISTANT_MODEL="gpt-4"

# Enable verbose output
export AI_ASSISTANT_VERBOSE="true"
```

### Task 1.7: Real-World Workflow Examples

Create examples of how Sarah would use this CLI in her daily work:

**Sarah's Marketing Consultant Workflows** (Complete these examples):

#### 1. Content Review Workflow
```bash
# Sarah's daily content review process
_[Write commands Sarah would use to review marketing content]_
```

#### 2. Client Email Automation
```bash
# Generating personalized client emails
_[Write commands for email generation workflow]_
```

#### 3. Research and Analysis
```bash
# Market research and trend analysis
_[Write commands for research workflow]_
```

#### 4. Batch Processing
```bash
# Processing multiple client reports
_[Write commands for batch processing]_
```

**Benefits for Sarah**:
- _[List specific benefits for Sarah's workflow]_
- _[How does this improve her productivity?]_
- _[What automation opportunities does this create?]_

---

## 🎯 Lab Summary and Reflection

### What You've Accomplished

In this lab, you've built a production-quality command-line interface that transforms Sarah's AI assistant into a professional tool. You've demonstrated:

- ✅ **Professional Argument Parsing**: Comprehensive CLI with subcommands, options, and help systems
- ✅ **Configuration Management**: Multi-source configuration with proper precedence and validation
- ✅ **Unix Conventions**: Proper exit codes, stdin/stdout handling, and error reporting
- ✅ **Error Handling**: Robust error management with user-friendly messages
- ✅ **Automation Support**: Pipeline integration and scripting capabilities
- ✅ **Professional Features**: Advanced capabilities like colored output, progress indicators, or shell completion

### Transformation Analysis

**Before**: Basic script with `input()` prompts  
**After**: Professional CLI tool with:
- Comprehensive help system
- Configuration management
- Multiple interface modes
- Automation support
- Professional error handling

### Real-World Impact for Sarah

Sarah can now:
- **Integrate with existing workflows**: Use the AI assistant in scripts and automation
- **Work efficiently**: Quick one-liner commands for common tasks
- **Collaborate**: Share CLI commands with team members
- **Scale operations**: Process multiple files and requests systematically
- **Maintain consistency**: Reliable configuration and repeatable processes

### Professional Skills Developed

- **CLI Design**: Understanding of professional command-line interface patterns
- **User Experience**: Creating tools that serve both casual and power users
- **System Integration**: Building tools that work well with existing Unix/Linux workflows
- **Configuration Management**: Handling multiple configuration sources professionally
- **Error Handling**: Providing helpful error messages and proper exit codes

### Next Steps

This professional CLI foundation prepares you for:
- Building web interfaces with Gradio (Lab 1.6)
- Creating hybrid applications with multiple interfaces
- Implementing comprehensive testing strategies
- Deploying tools for team and client use

**Congratulations!** You've created a professional CLI tool that follows industry best practices and serves real-world workflows. This is exactly the kind of tool that makes developers and power users productive and happy.